In [2]:
import tensorflow as tf
import pandas as pd 
import numpy as np 
import math 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler
import tensorflow as tf 
import seaborn  
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

In [3]:
df=pd.read_csv('fer2013.csv',delimiter=',')

In [4]:
x_train=[]
y_train=[]
x_test=[]
y_test=[]
for index,row in df.iterrows():
    k=row['pixels'].split(" ")
    if row['Usage']=='Training':
        x_train.append(np.array(k))
        y_train.append(row['emotion'])
    elif row['Usage']=='PublicTest':
        x_test.append(np.array(k))
        y_test.append(row['emotion'])

x_train=np.array(x_train,dtype ='uint8')
y_train=np.array(y_train,dtype ='uint8')
x_test=np.array(x_test,dtype ='uint8')
y_test=np.array(y_test,dtype ='uint8')

In [6]:
num_classes=7
y_train=to_categorical(y_train,num_classes=7)
y_test=to_categorical(y_test,num_classes=7)
x_train=x_train.reshape(x_train.shape[0],48,48,1)
x_test=x_test.reshape(x_test.shape[0],48,48,1)

In [7]:
datagen=ImageDataGenerator(
    rescale =1./255.0,
    rotation_range =10,
    horizontal_flip=True ,
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode ='nearest'
)
testgen=ImageDataGenerator(rescale=1.0/255.0)
datagen.fit(x_train) 
batch_size=64


In [8]:
train_flow=datagen.flow(x_train,y_train,batch_size=batch_size)
test_flow=testgen.flow(x_test,y_test,batch_size=batch_size)

In [9]:
import torch 
training_set =torch.FloatTensor(x_train)
test_set =torch.FloatTensor(y_train)


In [10]:
from keras.utils import plot_model
from keras.models import Sequential 
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import concatenate
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix

In [12]:
model=Sequential()
model.add(Conv2D(64,kernel_size =3,activation ='relu',padding ='same',name ='conv1_1',input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(Conv2D(64,kernel_size =3,activation ='relu',padding ='same',name ='conv1_2'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2,strides=2,name ='pool1_1'))
model.add(Dropout(0.3,name ='drop1_1'))

model.add(Conv2D(128,kernel_size =3,activation ='relu',padding ='same',name ='conv2_1'))
model.add(BatchNormalization())
model.add(Conv2D(128,kernel_size =3,activation ='relu',padding ='same',name ='conv2_2'))
model.add(BatchNormalization())
model.add(Conv2D(128,kernel_size =3,activation ='relu',padding ='same',name ='conv2_3'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2,strides=2,name='pool2_1'))
model.add(Dropout(0.3,name ='drop2_1'))

model.add(Conv2D(256,kernel_size =3,activation ='relu',padding ='same',name ='conv3_1'))
model.add(BatchNormalization())
model.add(Conv2D(256,kernel_size =3,activation ='relu',padding ='same',name ='conv3_2'))
model.add(BatchNormalization())
model.add(Conv2D(256,kernel_size =3,activation ='relu',padding ='same',name ='conv3_3'))
model.add(BatchNormalization())
model.add(Conv2D(256,kernel_size =3,activation ='relu',padding ='same',name ='conv3_4'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size =2,strides =2,name='pool3_1'))
model.add(Dropout(0.3,name ='drop3_1'))

model.add(Conv2D(256,kernel_size =3,activation ='relu',padding ='same',name ='conv4_1'))
model.add(BatchNormalization())
model.add(Conv2D(256,kernel_size =3,activation ='relu',padding ='same',name ='conv4_2'))
model.add(BatchNormalization())
model.add(Conv2D(256,kernel_size =3,activation ='relu',padding ='same',name ='conv4_3'))
model.add(BatchNormalization())
model.add(Conv2D(256,kernel_size =3,activation ='relu',padding ='same',name ='conv4_4'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2,strides=2,name='pool4_1'))
model.add(Dropout(0.3,name='drop4_1'))

model.add(Conv2D(512,kernel_size =3,activation ='relu',padding ='same',name ='conv5_1'))
model.add(BatchNormalization())
model.add(Conv2D(512,kernel_size =3,activation ='relu',padding ='same',name ='conv5_2'))

model.add(Conv2D(512,kernel_size =3,activation ='relu',padding ='same',name ='conv5_3'))
model.add(BatchNormalization())
model.add(Conv2D(512,kernel_size =3,activation ='relu',padding ='same',name ='conv5_4'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size =2,strides=2,name ='pool5_1'))
model.add(Dropout(0.3,name='drop5_1'))

model.add(Flatten(name ='flatten'))
model.add(Dense(units =128,activation='relu',name ='input'))
model.add(Dense(units=7,activation='softmax' ,name ='output'))



In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1_1 (Conv2D)            (None, 48, 48, 64)        640       
                                                                 
 batch_normalization_16 (Bat  (None, 48, 48, 64)       256       
 chNormalization)                                                
                                                                 
 conv1_2 (Conv2D)            (None, 48, 48, 64)        36928     
                                                                 
 batch_normalization_17 (Bat  (None, 48, 48, 64)       256       
 chNormalization)                                                
                                                                 
 pool1_1 (MaxPooling2D)      (None, 24, 24, 64)        0         
                                                                 
 drop1_1 (Dropout)           (None, 24, 24, 64)       

 batch_normalization_29 (Bat  (None, 3, 3, 512)        2048      
 chNormalization)                                                
                                                                 
 conv5_2 (Conv2D)            (None, 3, 3, 512)         2359808   
                                                                 
 conv5_3 (Conv2D)            (None, 3, 3, 512)         2359808   
                                                                 
 batch_normalization_30 (Bat  (None, 3, 3, 512)        2048      
 chNormalization)                                                
                                                                 
 conv5_4 (Conv2D)            (None, 3, 3, 512)         2359808   
                                                                 
 batch_normalization_31 (Bat  (None, 3, 3, 512)        2048      
 chNormalization)                                                
                                                                 
 pool5_1 (

In [14]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [14]:
model.fit(train_flow ,epochs=100,verbose=1,validation_data=test_flow)

Epoch 1/100
  3/449 [..............................] - ETA: 1:11:22 - loss: 3.9761 - accuracy: 0.1979

KeyboardInterrupt: 

In [9]:
from keras.models import model_from_json
import numpy as np
model = model_from_json(open("model.json", "r").read())
model.load_weights('model.h5')

In [10]:
import cv2
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [11]:
from keras.utils  import img_to_array

In [12]:
cap=cv2.VideoCapture(0) 
while cap.isOpened():
    res,frame=cap.read()
    height, width , channel = frame.shape#---------------------------------------------------------------------------
    sub_img = frame[0:int(height/6),0:int(width)]
    black_rect = np.ones(sub_img.shape, dtype=np.uint8)*0
    res = cv2.addWeighted(sub_img, 0.77, black_rect,0.23, 0)
    FONT = cv2.FONT_HERSHEY_SIMPLEX
    FONT_SCALE = 0.8
    FONT_THICKNESS = 2
    lable_color = (10, 10, 255)
    lable = "Emotion Detection"
    lable_dimension = cv2.getTextSize(lable,FONT ,FONT_SCALE,FONT_THICKNESS)[0]
    textX = int((res.shape[1] - lable_dimension[0]) / 2)
    textY = int((res.shape[0] + lable_dimension[1]) / 2)
    cv2.putText(res, lable, (textX,textY), FONT, FONT_SCALE, (0,0,0), FONT_THICKNESS)# prediction part --------------------------------------------------------------------------
    gray_image= cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_haar_cascade.detectMultiScale(gray_image )
    try:
        for (x,y, w, h) in faces:
            cv2.rectangle(frame, pt1 = (x,y),pt2 = (x+w, y+h), color = (255,0,0),thickness =  2)
            roi_gray = gray_image[y-5:y+h+5,x-5:x+w+5]
            roi_gray=cv2.resize(roi_gray,(48,48))
            image_pixels = img_to_array(roi_gray)
            image_pixels = np.expand_dims(image_pixels, axis = 0)
            image_pixels /= 255
            predictions = model.predict(image_pixels)
            max_index = np.argmax(predictions[0])
            emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
            emotion_prediction = emotion_detection[max_index]
            cv2.putText(res, "Sentiment: {}".format(emotion_prediction), (0,textY+22+5), FONT,0.7, lable_color,2)
            lable_violation = 'Confidence: {}'.format(str(np.round(np.max(predictions[0])*100,1))+ "%")
            violation_text_dimension = cv2.getTextSize(lable_violation,FONT,FONT_SCALE,FONT_THICKNESS )[0]
            violation_x_axis = int(res.shape[1]- violation_text_dimension[0])
            cv2.putText(res, lable_violation, (violation_x_axis,textY+22+5), FONT,0.7, lable_color,2)
    except:
            pass
    frame[0:int(height/6),0:int(width)] = res
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows

1/1 [==============================] - 0s 42ms/step


<function destroyAllWindows>

: 